In [3]:
!pip install sentencepiece
!pip install bert_score
!pip install spacy

     |████████████████████████████████| 6.2 MB 4.9 MB/s eta 0:00:01     |████▍                           | 849 kB 4.9 MB/s eta 0:00:02
     |████████████████████████████████| 454 kB 28.5 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 34.9 MB/s eta 0:00:01
     |████████████████████████████████| 636 kB 69.7 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.5 MB/s  eta 0:00:01
     |████████████████████████████████| 181 kB 35.5 MB/s eta 0:00:01
     |████████████████████████████████| 9.9 MB 28.1 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 7.8 MB/s  eta 0:00:01


In [4]:
import os
import sys
import math
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import spacy
import bert_score
from tqdm.notebook import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from datasets import load_metric

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using {device}")

using cuda


In [7]:
def get_sent_str(sentence_list):
    sent = " ".join(sentence_list)
    sent = re.sub(r" (?P<punc>[.?,])", r"\1", sent)
    return sent

def get_sent_list(sentences):
    sent_list = []
    for sent in sentences:
        sent_list.append(get_sent_str(sent))
    return sent_list

In [8]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

In [ ]:
print(tokenizer)

PreTrainedTokenizer(name_or_path='t5-base', vocab_size=32100, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>',

In [19]:
PRETRAINED_MODEL = 't5-base'
DIR = "question_generator/"
BATCH_SIZE = 1
SEQ_LENGTH = 128
EPOCHS = 100
USE_ANSWER = True
BEST = "toeflqa_finetune_summerize_withanswer.pt"
BEST_HF = "toeflqa_finetune_hf_summerize_withanswer"

# Check whether the specified path exists or not
if not os.path.exists(DIR):
    os.makedirs(DIR)

tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL)
tokenizer.add_special_tokens(
    {'additional_special_tokens': ['<answer>', '<context>']}
)
bertscore = load_metric("bertscore")

In [10]:
import importlib
util = importlib.import_module("data.TOEFL-QA.utils")


def set_fuzzy_context(key, raw_data):
    question = [raw_data[key]["question"]]
    results = []
    for ref in get_sent_list(raw_data[key]["sentences"]):
        results.append(bertscore.compute(predictions=question, references=[ref], lang='en'))
    idx = np.argsort(-1 * np.array([i["precision"] for i in results]).ravel())
    top5 = idx[:5]
    sent_list = get_sent_list(raw_data[key]["sentences"])
#         def get_surrounding(sent_list, ind):
#             if ind == 0:
#                 return set_list[i] + sent_list[i+1]
#             elif ind == len(sent_list)-1:
#                 return set_list[i-1] + sent_list[i]
#             else:
#                 return set_list[i-1] + sent_list[i] + sent_list[i+1]
    raw_data[key]["context"] = " ".join([sent_list[i] for i in sorted(top5)]) # reorder sentences

def preprocess(raw_data):
    keys = list(raw_data.keys())
    for key in tqdm(keys):
        raw_data[key]["question"] = get_sent_str(raw_data[key]["question"])
        raw_data[key]["answer"] = get_sent_str(raw_data[key]["answer"])
        raw_data[key]['sentences_list'] = get_sent_list(raw_data[key]['sentences'])
        
if "train_processed.npy" in os.listdir():
    train_raw = np.load("train_processed.npy", allow_pickle=True).item()
    dev_raw = np.load("dev_processed.npy", allow_pickle=True).item()
    test_raw = np.load("test_processed.npy", allow_pickle=True).item()
else:
    TOEFL_PATH = "./data/TOEFL-QA/data/"
    raw = util.load_data(TOEFL_PATH)
    train_raw, dev_raw, test_raw = tuple(raw)
    preprocess(dev_raw)
    np.save("dev_processed.npy", dev_raw)
    preprocess(test_raw)
    np.save("test_processed.npy", test_raw)
    preprocess(train_raw)
    np.save("train_processed.npy", train_raw)

### Problem
Context for both race and toefl texts / question contexts are not guaranteed to be <512 tokens, which is expected by our T5 base. Also, they both include lots of info not related to the question, which is the normal expectation in previous QG (ex. SQuAD models)
### Possible solutions
- Annotate.
- Use a metric (bertscore) to find semantically similar sentences to a given answer, and take the top n for a "fuzzy" context

In [12]:
class TOEFLDataset(Dataset):
    def __init__(self, data_dict):
        self.data = data_dict
        self.idx_map = list(data_dict.keys())       

    def __len__(self):
         return len(self.idx_map)

    def __getitem__(self, idx):   
        row = self.data[self.idx_map[idx]]
        
        sens4ques = dict() #token ids of sentence for each question
        sens4ques['input_ids'] = []
        sens4ques['attention_mask'] = []
        
        for sen in row['sentences_list']:
            #this is the input context for generating question (aka X)
            if USE_ANSWER:
                s = '<answer> ' + row['answer'] + ' <context> '+ sen
            else:
                s = sen
            encoded_text = tokenizer(
                s, 
                padding='max_length',
                max_length=SEQ_LENGTH,
                truncation=True,
                return_tensors="pt",
            )
            #print(encoded_text['input_ids'])
            sens4ques['input_ids'].append(encoded_text['input_ids'])
            sens4ques['attention_mask'].append(encoded_text['attention_mask'])

        sens4ques['input_ids'] = torch.vstack(sens4ques['input_ids']) # (num_sens, max_seq_length)
        sens4ques['attention_mask'] = torch.vstack(sens4ques['attention_mask']) #(num_sens, max_seq_length)

        #this is the target for generating question (aka Y, label)
        encoded_question = tokenizer(
            row['question'],
            padding=True,
            max_length=SEQ_LENGTH,
            truncation=True,
            return_tensors='pt',
        )
        encoded_question['input_ids'] = torch.squeeze(encoded_question['input_ids'])

        #return encoded_text.to(device), encoded_question.to(device)
        return sens4ques, encoded_question
    
train_set = TOEFLDataset(train_raw)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
dev_set = TOEFLDataset(dev_raw)
dev_loader = DataLoader(dev_set, batch_size=BATCH_SIZE)

In [13]:
train_set[0][1]

{'input_ids': tensor([  125,   405,     8,  5812,     3, 18531,    81,     8,   151,    16,
         4913,     3,  1258,   172, 18965,  5627,   116,    79,   708,  7954,
        15184,    11, 17941,     1]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [15]:
class MemNN(nn.Module):
    def __init__(self, vocab_size, embed_size, pad_id, hops=1, dropout=0.2, te=True, pe=True):
        super(MemNN, self).__init__()
        self.hops = hops
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.temporal_encoding = te
        self.position_encoding = pe
        self.pad_id = pad_id
        
        self.A = nn.ModuleList([nn.Embedding(self.vocab_size, self.embed_size, padding_idx=0)])
        nn.init.xavier_normal_(self.A[-1].weight.data)
        self.C = nn.ModuleList([nn.Embedding(self.vocab_size, self.embed_size, padding_idx=0)])
        nn.init.xavier_normal_(self.C[-1].weight.data)
        
        #weight tying, share parameters for A,C across layer
        for i in range(1, self.hops):
            self.A.append(self.A[-1])
            self.C.append(self.C[-1])
            
        #embedding for question
        self.B = nn.Embedding(self.vocab_size, self.embed_size, padding_idx=0)
        nn.init.xavier_normal_(self.B.weight.data)
        
        #output matrix
        #self.W = nn.Parameter(nn.init.xavier_normal_(torch.empty(self.embed_size, self.embed_size)))
        self.H = nn.Linear(self.embed_size, self.embed_size)
        nn.init.xavier_normal_(self.H.weight.data)
    
    def forward(self, sentences, query):
        #first get u_0, embedding for query (query_len, hidden_dim)
        query_len = query.size(0)
        query_pe = self.compute_weights(query_len) #position embedding weight metrixs for query (query_len, embed_size)
        u = self.B(query) #(query_len, embed_size)
        u = (u * query_pe).sum(0) #(embed_size)
        
        #sentences: (B, num_sens, max_seq_len)
        #print(sentences.shape)
        num_sens, max_seq_len = sentences.shape
        #get positional embeddign
        sens_pe = self.compute_weights(max_seq_len)
        
        for i in range(self.hops):
            m = self.A[i](sentences) # (num_sens, max_seq_len, embed_size)
            m *= sens_pe # (num_sens, max_seq_len, embed_size)
            m = torch.sum(m, 1) # (num_sens, embed_size)
            #ta = self.TA.repeat(1,1)[:num_sens, :]
            #m += ta
            
            c = self.C[i](sentences) # (num_sens, max_seq_len, embed_size)
            c *= sens_pe
            c = torch.sum(c, 1)
            #tc = self.TC.repeat(1, 1)[:num_sens, :] # (num_sens, embed_size)
            #c += tc
            
            p = torch.mm(m, u.unsqueeze(1)).squeeze(1) # (num_sens)
            p = F.softmax(p, dim=-1).unsqueeze(0) #(1, num_sens)
            o = torch.mm(p, c).squeeze(0) # o: (embed_size)
            u = o + self.H(u) #the updated query vector, 
        
        return p.squeeze(0) #p: (num_sens, ), p is the relative important score at the end 
            
    #return column vector for positional embedding within each sentence
    def compute_weights(self, J):
        d = self.embed_size
        func = lambda j, k: 1 - (j + 1) / J - (k + 1) / d * (1 - 2 * (j + 1) / J)
        weights = torch.from_numpy(np.fromfunction(func, (J, d), dtype=np.float16))
        #weights mi = sum_j(l_j * Ax_ij), j represent index in a sentence
        return weights.cuda() if torch.cuda.is_available() else weights

In [16]:
def train(epoch, best_val_loss):
    model.train()
    memnn.train()
    total_loss = 0.
    for batch_index, batch in tqdm(enumerate(train_loader)):
        data, target = batch 
        # data, target = (data.to(device), target.to(device))
        optimizer.zero_grad()

        #select the sentences using memory network
        data_input_ids = data['input_ids'].squeeze(0).to(device)
        data_attn_mask = data['attention_mask'].squeeze(0).to(device)
        target_input_ids = target['input_ids'].squeeze(0).to(device)
        
        sens_score = memnn(data_input_ids, target_input_ids)
        _, top_indices = sens_score.topk(5)
        top_data_input_ids = data_input_ids[top_indices] # (5, max_seq_length)

        #get rid of the padding value and re-process the input data
        top_data_input_ids = top_data_input_ids.reshape(-1) #(5 * max_seq_length)
        top_data_input_ids = top_data_input_ids[top_data_input_ids != 0] #get the rid of the padding (pad_idx = 0)
        top_data = tokenizer(
            tokenizer.decode(top_data_input_ids), 
            padding='max_length',
            max_length=512,
            truncation=True,
            return_tensors='pt',
        )
        top_data_input_ids = top_data['input_ids'].to(device)
        data_attn_mask = top_data['attention_mask'].to(device)

        masked_labels = mask_label_padding(target['input_ids']).to(device)
        output = model(
            input_ids=top_data_input_ids,
            attention_mask=data_attn_mask,
            labels=masked_labels,
        )
        output[0].backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        total_loss += output[0].item()
        if batch_index % 500 == 499:
            print(f'| epoch {epoch} | {batch_index}/{len(train_loader)} batches | loss {total_loss / 500}')
            total_loss = 0

def valid(epoch):
    model.eval()
    memnn.eval()
    total_loss = 0.
    with torch.no_grad():
        for batch_index, batch in tqdm(enumerate(dev_loader)):
            data, target = batch 
            # data, target = (data.to(device), target.to(device))
            optimizer.zero_grad()
            #select the sentences using memory network
            data_input_ids = data['input_ids'].squeeze(0).to(device)
            data_attn_mask = data['attention_mask'].squeeze(0).to(device)
            target_input_ids = target['input_ids'].squeeze(0).to(device)
            
            sens_score = memnn(data_input_ids, target_input_ids)
            _, top_indices = sens_score.topk(5)
            top_data_input_ids = data_input_ids[top_indices] # (5, max_seq_length)

            #get rid of the padding value and re-process the input data
            top_data_input_ids = top_data_input_ids.reshape(-1) #(5 * max_seq_length)
            top_data_input_ids = top_data_input_ids[top_data_input_ids != 0] #get the rid of the padding (pad_idx = 0)
            top_data = tokenizer(
                tokenizer.decode(top_data_input_ids), 
                padding='max_length',
                max_length=512,
                truncation=True,
                return_tensors='pt',
            )
            top_data_input_ids = top_data['input_ids'].to(device)
            data_attn_mask = top_data['attention_mask'].to(device)

            masked_labels = mask_label_padding(target['input_ids']).to(device)
            output = model(
                input_ids=top_data_input_ids,
                attention_mask=data_attn_mask,
                labels=masked_labels,
            )
            
            total_loss += output[0].item()
    return total_loss / len(dev_loader)
        
def mask_label_padding(labels):
    MASK_ID = -100
    labels[labels==tokenizer.pad_token_id] = MASK_ID
    return labels

In [17]:
config = T5Config(decoder_start_token_id=tokenizer.pad_token_id) # eos
model = T5ForConditionalGeneration(config).from_pretrained('t5-base')
model.resize_token_embeddings(len(tokenizer)) # to account for new special tokens
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [18]:
memnn = MemNN(config.vocab_size, config.hidden_size, pad_id=0, hops=1, dropout=0.2, te=True, pe=True)
memnn.to(device)

MemNN(
  (A): ModuleList(
    (0): Embedding(32128, 512, padding_idx=0)
  )
  (C): ModuleList(
    (0): Embedding(32128, 512, padding_idx=0)
  )
  (B): Embedding(32128, 512, padding_idx=0)
  (H): Linear(in_features=512, out_features=512, bias=True)
)

In [20]:
best_val_loss = float("inf")

for epoch in range(1, EPOCHS + 1):

    train(epoch, best_val_loss)
    torch.cuda.empty_cache()
    val_loss = valid(model)
    torch.cuda.empty_cache()
    print(f'\nend of epoch {epoch}\n valid loss: {val_loss}\n')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_loss': best_val_loss,
            'using_answer': USE_ANSWER
        }, DIR + BEST + ".best")
        model.save_pretrained(DIR + BEST_HF)
        print("Model saved.\n")
    if epoch % 10 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_loss': best_val_loss,
            'using_answer': USE_ANSWER
        }, DIR + BEST + f".epoch{epoch}")
        model.save_pretrained(DIR + BEST_HF + f".epoch{epoch}")


/home/jiayueg/miniconda3/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:185: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


| epoch 1 | 499/717 batches | loss 4.935819515228271





end of epoch 1
 valid loss: 4.2939108033334055

Model saved.



| epoch 2 | 499/717 batches | loss 4.4045761089324955





end of epoch 2
 valid loss: 4.143870682485642

Model saved.



...



end of epoch 98
 valid loss: 1.9952678483340047



| epoch 99 | 499/717 batches | loss 2.290180093109608





end of epoch 99
 valid loss: 1.9905611184095182

Model saved.



| epoch 100 | 499/717 batches | loss 2.2546058756113054





end of epoch 100
 valid loss: 1.9891924550456386

Model saved.

